In [139]:
import matplotlib.pyplot as plt  
# iscrtavanje slika i plotova unutar samog browsera
%matplotlib inline 

import matplotlib.pylab as pylab
# prikaz vecih slika 
pylab.rcParams['figure.figsize'] = 12,10


import numpy as np
import cv2 # OpenCV biblioteka
import math

from collections import namedtuple
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')


In [144]:
def detectLines(img):
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150, apertureSize = 3)

    minLineLength = 200
    maxLineGap = 13
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, np.array([]), minLineLength, maxLineGap)

    return lines

def lengthOfLine(x1, x2, y1, y2):
    
    length = math.sqrt(math.pow((x1 - x2), 2) + math.pow((y1 - y2), 2))
    return length

def getLeftAndRightPoint(x1, x2, y1, y2):
    
    rightPoint = []
    leftPoint = []
    
    if x1 - x2 < 0:
        leftPoint.append(x1)
        rightPoint.append(x2)
    else:
        leftPoint.append(x2)
        rightPoint.append(x1)
    
    if y1 - y2 < 0:
        leftPoint.append(y2)
        rightPoint.append(y1)
    else:
        leftPoint.append(y1)
        rightPoint.append(y2)
        
    return rightPoint, leftPoint 

def isRightLine(x1, x2, y1, y2, newLines):
    
    rightPoint, leftPoint = getLeftAndRightPoint(x1, x2, y1, y2)
    
    for newLine in newLines:
        for a1,b1,a2,b2 in newLine: 
            newRightPoint, newLeftPoint = getLeftAndRightPoint(a1, a2, b1, b2)
            
            if lengthOfLine(rightPoint[0], newRightPoint[0], rightPoint[1], newRightPoint[1]) <= 50:
                return True
    return False

def getMinLenLine(lines):
    
    minLen = 200000
    minLine = None
    for line in lines:
        for a1,b1,a2,b2 in line: 
            rightPoint, leftPoint = getLeftAndRightPoint(a1, a2, b1, b2)
            
            length = lengthOfLine(rightPoint[0], leftPoint[0], rightPoint[1], leftPoint[1])
            if length <= minLen:
                minLen = length
                minLine = line
                
    return minLine

def getFirstLine(lines):
    
    firstLines = []
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if len(firstLines) == 0:
                firstLines.append(line)
            else:
                if isRightLine(x1, x2, y1, y2, firstLines) is True:
                    firstLines.append(line)
                    
    
    rightLine = getMinLenLine(firstLines)
    return rightLine

    

def getSecondLine(lines, firstLine):
    
    secondLines = []
    newRightPoint, newLeftPoint = getLeftAndRightPoint(firstLine[0], firstLine[2], firstLine[1], firstLine[3])
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            rightPoint, leftPoint = getLeftAndRightPoint(x1, x2, y1, y2)
            
            if lengthOfLine(rightPoint[0], newRightPoint[0], rightPoint[1], newRightPoint[1]) >= 50:
                secondLines.append(line)
                    
    
    rightLine = getMinLenLine(secondLines)
    return rightLine
            
        
def getRightLines(lines):
    
    line1 = getFirstLine(lines)
    line2 = getSecondLine(lines, line1[0])
    
    newLines = [line1, line2]
    return newLines

def getAddAndSubLine(lines):
    
    line1 = lines[0][0]
    line2 = lines[1][0]
    
    right1, left1 = getLeftAndRightPoint(line1[0], line1[2], line1[1], line1[3])
    right2, left2 = getLeftAndRightPoint(line2[0], line2[2], line2[1], line2[3])
    
    if right1[1] - right2[1] > 0:
        return line2, line1 # addLine, subLine
    else:
        return line1, line2 # addLine, subLine
    
def crossAddLine(x, y, w, n, k, rightAddPoint, leftAddPoint):
    
    y1=k*x+n
    d = abs(k * x + (-1) * y + n) / float(math.sqrt(k * k + 1))
#     if d <= 6:
#         print('Rastojanje gore-levo: %f' % d)

#         print('\nRastojanje gore-levo: %f' % d)
#         print('y1-y: %d' % (y1 - y))
#         print('X: %f\tY: %f' % (x,y))

    if y1 - y < 0 and d < 10 and d >= 7 and x >= leftAddPoint[0] and x <= rightAddPoint[0] + w / 2:
        return True
    else:
        return False
    
def crossSubLine(x, y, w, h, n, k, rightSubPoint, leftSubPoint, numFrame):
    
    y1=k*x+n
    d = abs(k * x + (-1) * y + n) / float(math.sqrt(k * k + 1))
    
    if d <= 10 and numFrame == 0 and x >= leftSubPoint[0] - w / 8 and x <= rightSubPoint[0] + math.ceil(float(w) / 4):
        return True
        
    
    if y1 - y > 0 and d < 7 and d >= 3 and x >= leftSubPoint[0] - w / 8 and x <= rightSubPoint[0] + math.ceil(float(w) / 4):
        return True
    else:
        return False
    
def fillAddArea(contours, n, k, rightAddPoint, leftAddPoint, points):
        
    for contour in contours:
        
        x,y,w,h = cv2.boundingRect(contour)
        
        y1=k*x+n
        d = abs(k * x + (-1) * y + n) / float(math.sqrt(k * k + 1))
    
        if y1 - y < 0 and d < 10 and d >= 7 and x >= leftAddPoint[0] and x <= rightAddPoint[0] + w / 2:
             
            if isExistNearPoint(x, y, d, n, k, points):
                points.append([x, y, True, 2, 2])
                
            elif not isExistPoint(x, y, points):
                points.append([x, y, False, 1, 2])
                
def fillSubArea(contours, n, k, rightSubPoint, leftSubPoint, points, numFrame):
        
    for contour in contours:
        
        x,y,w,h = cv2.boundingRect(contour)
        x += w
        y += h
        
        y1=k*x+n
        d = abs(k * x + (-1) * y + n) / float(math.sqrt(k * k + 1))
    
        if y1 - y > 0 and d < 7 and d >= 3 and x >= leftSubPoint[0] - w / 8 and x <= rightSubPoint[0] + math.ceil(float(w) / 4):
            pom = False
                
            if isExistNearPoint(x, y, d, n, k, points):
                points.append([x, y, True, 2, 2])
                
            elif not isExistPoint(x, y, points):
                points.append([x, y, False, 1, 2])
                
        elif d <= 10 and numFrame == 0 and x >= leftSubPoint[0] - w / 8 and x <= rightSubPoint[0] + math.ceil(float(w) / 4):
            pom = False
                
            if isExistNearPoint(x, y, d, n, k, points):
                points.append([x, y, True, 2, 2])
                
            elif not isExistPoint(x, y, points):
                points.append([x, y, False, 1, 2])

        
def isExistPoint(x, y, points):
    
    for point in points:
        
        if point[0] == x and point[1] == y:
            point[3] += 1 #broj pojavljivanja povecavam za jedan
            #print('Vec postoji')
            return True
        
    return False

def isExistNearPoint(x, y, d, n, k, points):
    
    for point in points:
        
        d_point = abs(k * point[0] + (-1) * point[1] + n) / float(math.sqrt(k * k + 1))
        if point[0] != x or point[1] != y:
            if abs(d - d_point) <= 4 and abs(x - point[0]) <= 8 and abs(y - point[1]) <= 8:
                #print('Postoji blizi')
                return True
        
    return False

def isFirstTime(x, y, points):
    
    for point in points:
        
        if point[0] == x and point[1] == y:
            
            point[2] = True
            if point[3] == 1:
                
                point[3] += 1
                return True
    return False

def erasePoints(points):
    
    for point in points:
        
        if point[2] is False:
            
            point[4] -= 1
            if point[4] == -1:
                points.remove(point)
            
def setPoints(points):
    
    for point in points:
        point[2] = False


def getNewContours(contours):
    
    newContours = []
    for contour in contours: # za svaku konturu
        area = cv2.contourArea(contour) # pronadji pravougaonik minimalne povrsine koji ce obuhvatiti celu konturu

        if area > 60 :
            newContours.append(contour)
            #print('Povrsina konture: %d' % area)
    return newContours

def getNumber(out):
    
    num = 0
    for i in range(1, len(out)):
        if out[i] > out[num]:
            num = i
    return num

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    
def isDigitsOverlap(digit, digits, typeOfLine):
    
    x,y,w,h = cv2.boundingRect(digit)
    ra = Rectangle(x - 4, y - 4, x + w + 4, y + h + 4)
    
    for d in digits:
        
        x1,y1,w1,h1 = cv2.boundingRect(d)
        
        if x == x1 and y == y1 and w == w1 and h == h1:
            continue
            
        rb = Rectangle(x1 - 4, y1 - 4, x1 + w1 + 4, y1 + h1 + 4)
        
        result = area(ra, rb)
        
        if result != None:
            if result > 50 and typeOfLine == "ADD":
                return True

            elif result > 100 and result < 130 and typeOfLine == "SUB":
                return True
        
    return False

def findNumber(digit, frames, n, k):
    
    x,y,w,h = cv2.boundingRect(digit)
    
    for frame in reversed(frames):
        
        
        for contour in frame[0]:
            
            x1,y1,w1,h1 = cv2.boundingRect(contour)
            

            yn=k*(x + w)+n
            d1 = abs(k * (x + w) + (-1) * (y + h) + n) / float(math.sqrt(k * k + 1))
            #d2 = abs(k * x1 + (-1) * y1 + n) / float(math.sqrt(k * k + 1))

            if abs(x - x1) <= 20 and abs(y - y1) <= 10 and d1 > 35 and yn - (y + h) > 0:
                
                x = x1
                y = y1
                w = w1
                h = h1
                
                ra = Rectangle(x - 4, y - 4, x + w + 4, y + h + 4)
                
                overlap = False
                
                for d in frame[0]:
                    
                    x1,y1,w1,h1 = cv2.boundingRect(d)
    
                    if x == x1 and y == y1:
                        continue
            
                    rb = Rectangle(x1 - 4, y1 - 4, x1 + w1 + 4, y1 + h1 + 4)
        
                    result = area(ra, rb)
        
                    if result != None:
                        if result >= 30:
                            overlap = True

                if overlap == False:
                    return contour, frame[1]
        
                break;
        #print('PROLAZI KROZ LINIJU')
        x -= 1
        y -= 1

    return None, None
    

In [ ]:
# Larger CNN for the MNIST Dataset
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.layers.convolutional import MaxPooling2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils import np_utils
from keras import backend as K
K.set_image_dim_ordering('th')
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][pixels][width][height]
X_train = X_train.reshape(X_train.shape[0], 1, 28, 28).astype('float32')
X_test = X_test.reshape(X_test.shape[0], 1, 28, 28).astype('float32')
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

# define the larger model
def larger_model():
    
    # create model
    model = Sequential()
    
    
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (1, 28, 28)))
    model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2)))
    model.add(Dropout(0.25))


    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
    model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(Dropout(0.25))


    model.add(Flatten())
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation = "softmax"))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# build the model
model = larger_model()


# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=200)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Large CNN Error: %.2f%%" % (100-scores[1]*100))


In [78]:
from keras.models import model_from_json

# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [147]:
fileOut = open('out.txt', 'w')
fileOut.write("SW 41/2014 Momir Kostic" + '\n' + "file" + '\t' + "sum" + '\n')

In [ ]:
for videoNum in range(0, 10):
    
    cap = cv2.VideoCapture('videos/video-' + str(videoNum) + '.avi')
    
    gray = None
    numFrame = 0
    cap.set(1, 0);
    suma = 0
    addAreaPoints = [] #model [x, y, inArea, repeat, number]
    subAreaPoints = []
    frames = []
    
    
    while(cap.isOpened()):
        ret, frame = cap.read()

        if ret is False:
            break

        if True:  
#         if numFrame >= 110:

            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

            if numFrame == 0:
                lines = detectLines(frame)
                lines = getRightLines(lines)

                addLine, subLine = getAddAndSubLine(lines)

                k1 = (addLine[1] - addLine[3]) / float(addLine[0] - addLine[2])
                n1 = addLine[1] - k1 * addLine[0]

                k2 = (subLine[1] - subLine[3]) / float(subLine[0] - subLine[2])
                n2 = subLine[1] - k2 * subLine[0]


                rightAdd, leftAdd = getLeftAndRightPoint(addLine[0], addLine[2], addLine[1], addLine[3])
                rightSub, leftSub = getLeftAndRightPoint(subLine[0], subLine[2], subLine[1], subLine[3])


            for line in lines:
                for x1,y1,x2,y2 in line:
                    cv2.line(gray, (x1, y1), (x2, y2), (255, 255, 255), 2)

            if True:
                            
                try:
        
                    lb = np.array([150, 150, 150], np.uint8)
                    ub = np.array([255, 255, 255], np.uint8)
                
                    img = cv2.inRange(frame, lb, ub)
                except:
                    break


                imgNumber = cv2.dilate(img, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2)), iterations=2)
                
                
                    # # Find contours in the image
                img, contours, hierarchy = cv2.findContours(imgNumber.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                newContours = getNewContours(contours)
                
                fillAddArea(newContours, n1, k1, rightAdd, leftAdd, addAreaPoints)
                fillSubArea(newContours, n2, k2, rightSub, leftSub, subAreaPoints, numFrame)
                
                if len(frames) < 150:
                    frames.append([newContours, imgNumber])
                else:
                    frames.pop(0)
                    frames.append([newContours, imgNumber])

                for contour in newContours:
                
                    #print('Usao')
                    x,y,w,h = cv2.boundingRect(contour)
                    # Draw the rectangles
                    #cv2.rectangle(imgNumber, (x,y),(x+w,y+h), (255, 0, 0), 2)
                    pom = False
                    if numFrame >= 10 and crossAddLine(x, y, w, n1, k1, rightAdd, leftAdd) is True and isFirstTime(x, y, addAreaPoints) is True:
                        
                        pom = True
                        # Make the rectangular region around the digit
                        leng = int(h * 1.6)
                        pt1 = int(y + h // 2 - leng // 2)
                        pt2 = int(x + w // 2 - leng // 2)

                        if pt1 < 0 or pt2 < 0 :
                            continue;
                            
                        isOverlap = False
                            
                        if isDigitsOverlap(contour, newContours, "ADD"):
                            #print('----------Ima poklapanja - ADD-------------')
                            
                            digit, oldImage = findNumber(contour, frames, n1, k1)
                            
                            if digit != None:
                                x,y,w,h = cv2.boundingRect(digit)
#                                 print('IZ GLAVNE FUNKCIJE:  X = %d  Y = %d' % (x, y))
                                roi = oldImage[y - 2:y + 2 + h, x - 3: x + 3 + w]
                                isOverlap = True
                                


                        #roi = imgNumber[pt1:pt1+leng, pt2:pt2+leng]
                        if isOverlap == False:
                            roi = imgNumber[y - 3:y + 3 + h, x - 3: x + 4 + w]

                        
#                         roi = cv2.dilate(roi, (2, 2))
#                         roi = cv2.erode(roi, (3, 3))
                        

                        # Resize the image
                        roi = cv2.resize(roi,(28,28), interpolation = cv2.INTER_AREA)


                        
                        roi = cv2.erode(roi, (2, 2))
                        #roi = cv2.erode(roi, (1, 1))
#                         roi = cv2.dilate(roi, (2, 2))

                        #roi = cv2.dilate(roi, (3, 3))
                                # Calculate the HOG features
                        #roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
                        roi = np.expand_dims(roi, axis = 0)
                        nbr = loaded_model.predict(np.array([roi], 'float64'))[0] #[[]]
                        
                        number = getNumber(nbr)
                        
                        d = abs(k1 * x + (-1) * y + n1) / float(math.sqrt(k1 * k1 + 1))
#                         print('Rastojanje gore-levo: %f' % d)
#                         print('X: %f\tY: %f' % (x,y))
                        #print('Sabiranje broja: %d' % number)
                        suma += int(number)
        
                    if crossSubLine(x + w, y + h, w, h, n2, k2, rightSub, leftSub, numFrame) is True and isFirstTime(x + w, y + h, subAreaPoints) is True:
                           
                        if not pom:
                            # Make the rectangular region around the digit
                            leng = int(h * 1.6)
                            pt1 = int(y + h // 2 - leng // 2)
                            pt2 = int(x + w // 2 - leng // 2)

                            if pt1 < 0 or pt2 < 0 :
                                continue;
                                
                            isOverlap = False
                            
                            if isDigitsOverlap(contour, newContours, "SUB"):
                                #print('----------Ima poklapanja - SUB -------------')
                            
                                digit, oldImage = findNumber(contour, frames, n2, k2)
                            
                                if digit != None:
                                    x,y,w,h = cv2.boundingRect(digit)
#                                 print('IZ GLAVNE FUNKCIJE:  X = %d  Y = %d' % (x, y))
                                #imgNumber = oldImage
#                                 img = cv2.rectangle(oldImage, (x - 5,y - 5), (x+w + 5,y+h + 5), (255,255,255), 2)
                                    roi = oldImage[y - 2:y + 2 + h, x - 3: x + 3 + w]
                                    isOverlap = True
                                
                                
                            if isOverlap == False:
                                roi = imgNumber[y - 2:y + 2 + h, x - 3: x + 3 + w]

#                             roi = cv2.dilate(roi, (2, 2))
    #                         roi = cv2.erode(roi, (3, 3))

                            # Resize the image
                            roi = cv2.resize(roi,(28,28), interpolation = cv2.INTER_AREA)



                            roi = cv2.erode(roi, (2, 2))
                            #roi = cv2.dilate(roi, (3, 3))
                                    # Calculate the HOG features
                            #roi_hog_fd = hog(roi, orientations=9, pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
                            roi = np.expand_dims(roi, axis = 0)
                            nbr = loaded_model.predict(np.array([roi], 'float64'))[0] #[[]]

                            number = getNumber(nbr)
                        #print('Oduzimanje broja: %d' % number)
                        suma -= int(number)
                        

                erasePoints(addAreaPoints)
                erasePoints(subAreaPoints)
                setPoints(addAreaPoints)
                setPoints(subAreaPoints)



        numFrame += 1
    print('Suma%d: %d\n' % (videoNum,suma))
    fileOut.write("video-" + str(videoNum) + ".avi" + '\t' + str(suma) + '\n')
    cv2.destroyAllWindows()


fileOut.close()